In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import astropy.units as u
import scipy
from astropy import constants as const
from scipy import optimize
import scipy.integrate as integrate
from scipy.integrate import quad 
from scipy.special import *
import math
from ReadFile import Read
import pandas as pd

In [2]:
Mgsol = 5.08 #banda g
Misol = 4.53 #banda i
Mrsol = 4.64 #banda r 

In [3]:
#################### READ CATALOGUE ########################

# col_names = ["objid", "specobjid", "z", "nsersic_g", "Re_g", "L_g", "ba_g", "Mg", "nsersic_r", "Re_r", "L_r", "ba_r", "Mr", "nsersic_i", "Re_i", "L_i", "ba_i", "Mi", "Vol_max", "P_early"]
# catalog = pd.read_csv("SDSS_DR7_sersic_photometric_catalog2.txt", dtype= None, sep='\s+', names=col_names)

In [4]:
# catalog
catalog =  Read('SDSS_DR7_sersic_photometric_catalog2.txt')

In [5]:
# catalog.to_csv('SDSS_DR7.csv')
catalog

array([(587722952230174976,  96486454284255232, 0.0302843, 1.7263,  1.7026 , 1.43027e+09, 0.3388, -17.8085, 1.6529,  1.91125, 1.54479e+09, 0.3366, -18.1989, 1.7017,  1.96545, 1.67209e+09, 0.3474, -18.5282, 2.41528e+06, 0.153371 ),
       (587722952230174976,  96486454292643840, 0.0779226, 0.4261,  5.74619, 1.29350e+10, 0.3803, -20.1994, 0.699 ,  5.25198, 1.29608e+10, 0.3867, -20.5591, 0.8582,  5.22381, 1.42117e+10, 0.3845, -20.8516, 5.58970e+07, 0.20661  ),
       (587722952230175104,  96486454296838144, 0.160228 , 0.8707, 15.9537 , 8.05371e+10, 0.5794, -22.185 , 1.433 , 15.3929 , 9.02321e+10, 0.5795, -22.5166, 1.5128, 13.9292 , 9.90569e+10, 0.5938, -22.9597, 4.23392e+08, 0.0523307),
       ...,
       (588848901543690496, 151938424012537856, 0.10459  , 5.156 , 19.0088 , 1.28232e+11, 0.8587, -22.69  , 5.2066, 17.7542 , 1.81702e+11, 0.8565, -22.9481, 5.5003, 18.3337 , 2.31359e+11, 0.8572, -23.8807, 4.19699e+08, 0.92721  ),
       (588848901543690496, 151938423832182784, 0.126815 , 3.128

In [6]:
########## LUMINOSITES INTENSITIES AND MAGNITUDES ##################


# Sersic general equation for any n. Eq. 2.22 from Galaxy Formation & Evolution 
# Need to define Reff first
# Bn 2n-0.324 (but only for n=>1) (B beta)
# To write I(R) in terms of L_tot instead of I0:
#         I(R)= {(L_tot*bn**2n)/(2*pi*Re**2*G(2n))} exp[-bn(R/Re)**(1/n)]
# To write I(R) in terms of L_tot instead of Ie:
#         I(R)= {(L_tot*bn**2n)/(2*pi*Re**2*G(2n)*exp(bn))} exp[-bn{(R/Re)**(1/n)-1}]


def I0_to_Ie(I0):
    #units Lsun / kpc**2
    
    Bn = 2*n-0.324 
    
    return I0/np.exp(Bn)

def Ie_to_I0(Ie):
    #units Lsun / kpc**2
    
    Bn = 2*n-0.324 
    
    return Ie*np.exp(Bn)
    
    

def Ltot_to_I0(Ltot, Re, n):
    # Function that transform a given luminosity to I0 from eq 2.22 and 2.24
    # Takes in total luminosity, effective radius and sersic index
    # Returns I0
    # Ltot*(Bn)**(2n) / Re**2/(2*np.pi*n)/gamma
    #units Lsun / kpc**2
    
    Bn = 2*n-0.324
    
    gm = gamma(2*n)  #gamma function
    
    I0 = Ltot*(Bn)**(2*n) / Re**2/(2*np.pi*n)/gm
    
    return I0

def Ltot_to_Ie(Ltot, Re, n):
    # Function that transform a given luminosity to I0 from eq 2.22 and 2.24
    # Takes in total luminosity, effective radius and sersic index
    # Returns Ie
    # Ltot*(Bn)**(2n) / Re**2/(2*np.pi*n)/gamma/np.exp(Bn)
    
    Bn = 2*n-0.324
    
    gm = math.gamma(2*n)  #gamma function
    
    Ie = Ltot*(Bn)**(2*n) / Re**2/(2*np.pi*n)/gm/np.exp(Bn)
    
    return Ie
    

def sersic_profile(r, Ie, n, Re):
    # I0 puede ser L_tot
    # The surface brightness profile of spheroidal galaxies is generally well fit by the Sersic profile
    # Compare to sersic mass density 
    # I(R) = I0*exp[-Bn(R/Re)**(1/n)] = Ie*exp[-Bn{(R/Re)**(1/n)-1}]
    # Ie = I0/exp(Bn)
    # Inputs: radius, Intesity, sersic index, Reff
    # units are Lsun / kpc**2
    # Eq 2.22 from Galaxy Formation & Evolution
   
    Bn = 2*n-0.324
    ratio = (r/Re)**(1/n)
    exponent = -Bn*(ratio-1)
    
    return Ie*np.exp(exponent)



def L_profile(r, Ie, n, Re):
    # Total luminosity. Eq. 2.24 from Galaxy Formation & Evolution 
    # The total luminosity of a spherical system but as a function of radius. 
    # Compare to mass_prof_disk/sph from old code 
    # L = 2*pi*integral(I(R)RdR)
    # integrate sersic profile from 0 to R
    # inputs:
    #        radius, Intensity(e), sersic index, R eff
    
    def L(r, Ie, n, Re):
        
        return 2*np.pi*sersic_profile(r, Ie, n, Re)*r
    
    I = integrate.quad(L, 0 ,r, args=(Ie, n, Re))
    
    ansI = I[0]
    
    return ansI

def L_tot(Ie, n, Re):
    # Total luminosity. Eq. 2.24 from Galaxy Formation & Evolution 
    # The total luminosity of a spherical system with a Sersic profile is given by this equation.
    # Compare to mass_prof_disk/sph from old code 
    # Inputs :
    #         Intensity(e), sersic index, Eff rad
    # L = 2*pi*integral(I(R)RdR) = (2*pi*G(2n)/(Bn)**(2*n))*I0*Re**2
    # integrate sersic profile from 0 to inf
    # Returns the total Luminosity
    "units of Lsun?"
    
    Bn = 2*n-0.324
    
    gm = math.gamma(2*n)  #gamma function buscar funcion completa
    
    I0 = Ie*np.exp(Bn)    
    
    L = (2*np.pi*n*gm/(Bn)**(2*n))*I0*Re**2
    
    return L

def abs_mag_to_log10Lx(Mabs_x, Mabs_xsol):
#     Input: x = band, Mx = absolute magnitude of a given object at a given band
#     Returns the total luminosity at a given band of the galaxy
#     M_gsol = 5.11 #banda g 
#     M_isol = 4.53 #banda i
#     Log(L_x/L_xsol) = -0.4* (M_x - M_xsol) 
#     Eq 2.6 from Galaxy Formation & Evolution
#     This returns the LOG of the luminosity!
    
    return -0.4*(Mabs_x-Mabs_xsol) #### Log(Lx/L_xsol)


def Lx_to_abs_mag(Lx, Mabs_xsol):
    # M_x  =  -2.5 *log(L_x/L_xsol) + M_xsol
    # It takes L_profile as input to make it an absolute magnited as a function of radius
    # Substract two different bands to get the color
    

    return -2.5 * np.log10(Lx) + Mabs_xsol #### Absolute Magnitude

In [7]:
 ################ MASS PROFILES #########################   
    
    
def stellar_mass_profile(func, r, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol):
    
#   "units of solar mass/kpc?"
# Log(Ms/Msol) = -0.68 + 0.73 * (g-i)  + log10(L_i/L_isol)
# To make it a function of radius
# Inputs:
#       radius, Intensity(e), sersic index, Magnitudes of the g and i bands
   
   
    Lr_x = L_profile(r, Ie_x, n_x, Re_x)
    Lr_y = L_profile(r, Ie_y, n_y, Re_y)
    Mr_x = Lx_to_abs_mag(Lr_x, Mabs_xsol)
    Mr_y = Lx_to_abs_mag(Lr_y, Mabs_ysol)
   # We need the luminosity profile of the i band to make it a function of radius 
   
    return func(Lr_y, Mr_x, Mr_y) ##llamar una funcion cualquier (ie func en el input) func puede ser Stellar_mass_from_gi



def Stellar_mass_from_gi(LtI, Mxg, Mxi):
# Log(Ms/Msol) = -0.68 + 0.73 * (g-i)  + log10(L_i/L_isol)
# Total stellar mass 
# Instead of the Luminosity profile, we use the total luminosity
#"only units of solar mass?"
   
    color = Mxg - Mxi 
   
    logMs =  -0.68 + 0.73 * color  + np.log10(LtI)
    
   
    return 10**logMs


def Stellar_mass_Bell_gr(LtR, Mxg, Mxr):
   #log10Mste_Bell_r = -0.406 + 1.097 * (g-r) + log10(L_r) 
   
    color = Mxg - Mxr
   
    logMs = -0.406 + 1.097 * color + np.log10(LtR) 
   
    return 10**logMs

def Stellar_mass_Bell_gi(LtI, Mxg, Mxi):
   #log10Mste_Bell_i = - 0.252 + 0.518*(g-i)+log10(L_i)
   
    color = Mxg - Mxi
   
    logMs = - 0.252 + 0.518* color + np.log10(LtI)
   
    return 10**logMs

def Stellar_mass_Zibetti_gr(LtR, Mxg, Mxr):
   #log10Mste_Zibetti_r = -0.840 + 1.654 * (g-r) + log10(L_r)
   
    color = Mxg - Mxr
   
    logMs = -0.840 + 1.654 * color + np.log10(LtR) 
   
    return 10**logMs

def Stellar_mass_Zibetti_gi(LtI, Mxg, Mxi):
   #log10Mste_Zibetti_i = -0.963 + 1.032*(g-i) + log10(L_i)
   
    color = Mxg - Mxi
   
    logMs = -0.963 + 1.032 * color + np.log10(LtI)
   
    return 10**logMs


def r_effective(func, Mstot,Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol):
   # Computes the effective (mass) radius. Where half of the total mass is located
   #  total_stellar_mass/2 = stellar_mass_profile 

    def f(r, func, Mstot, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol):
        
       
        return 0.5* Mstot - stellar_mass_profile(func, r, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol)
    
    fa = f(Re_y/6, func, Mstot, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol)
    
    fb = f(Re_y*6, func, Mstot, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol)
     
    if fa > 0 and fb < 0:
        
        r_m2 = optimize.bisect(f, Re_y/6, Re_y*6, args=(func, Mstot, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol))
        
    elif fa < 0 and fb > 0:
        
        r_m2 = optimize.bisect(f, Re_y/6, Re_y*6, args=(func, Mstot, Ie_x, Ie_y, n_x, n_y, Re_x, Re_y, Mabs_xsol, Mabs_ysol))
    
    else:
        
        r_m2 = - 99 ## not a physical number. means its not calculating 
           
  
    return r_m2

In [8]:
# calcular mass_tot con las nuevas funciones
# calcular R_eff (func y Mstot vienen de la misma)

# exctracting the values I need from the table 

n_g = catalog['nsersic_g']
Re_g = catalog['Re_g']
Ltot_g = catalog['L_g']
Mg = catalog['Mg']

n_i = catalog['nsersic_i']
Re_i = catalog['Re_i']
Ltot_i = catalog['L_i']
Mi = catalog['Mi']

n_r = catalog['nsersic_r']
Re_r = catalog['Re_r']
Ltot_r = catalog['L_r']
Mr = catalog['Mr']

disk = np.where(catalog['P_early'] < 0.65)
sphr = np.where(catalog['P_early'] >= 0.65)

In [9]:
sphr

(array([    10,     16,     22, ..., 603855, 603858, 603861]),)

In [10]:
# n_g = 1.7263
# Re_g = 1.7026
# Ltot_g = 1.43027e+09
# Mg = -17.8085

# n_r = 1.6529
# Re_r = 1.91125
# Ltot_r = 1.54479e+09
# Mr =  -18.1989

# n_i = 1.7017
# Re_i = 1.96545
# Ltot_i = 1.67209e+09
# Mi = -18.5282



# Ie_g = Ltot_to_Ie(Ltot_g, Re_g, n_g)
# Ie_i = Ltot_to_Ie(Ltot_i, Re_i, n_i)
# Ie_r = Ltot_to_Ie(Ltot_r, Re_r, n_r)
# msgi = Stellar_mass_from_gi(Ltot_i, Mg, Mi)
# msgr_B = Stellar_mass_Bell_gr(Ltot_r, Mg, Mr)
# msgi_B = Stellar_mass_Bell_gi(Ltot_i, Mg, Mi)
# msgr_Z = Stellar_mass_Zibetti_gr(Ltot_r, Mg, Mr)
# msgi_Z = Stellar_mass_Zibetti_gi(Ltot_i, Mg, Mi)



# reffmsgi = r_effective(Stellar_mass_from_gi, msgi, Ie_g, Ie_i, n_g, n_i, Re_g, Re_i, Mgsol, Misol)
# reffmsgr_B = r_effective(Stellar_mass_Bell_gr, msgr_B, Ie_g, Ie_r, n_g, n_r, Re_g, Re_r, Mgsol, Mrsol)
# reffmsgi_B = r_effective(Stellar_mass_Bell_gi, msgi_B, Ie_g, Ie_i, n_g, n_i, Re_g, Re_i, Mgsol, Misol)
# reffmsgr_Z = r_effective(Stellar_mass_Zibetti_gr, msgr_Z, Ie_g, Ie_r, n_g, n_r, Re_g, Re_r, Mgsol, Mrsol)
# reffmsgi_Z = r_effective(Stellar_mass_Zibetti_gi, msgi_Z, Ie_g, Ie_i, n_g, n_i, Re_g, Re_i, Mgsol, Misol)

# print(np.log10(msgi),reffmsgi)
# print(np.log10(msgr_B),reffmsgr_B)
# print(np.log10(msgi_B),reffmsgi_B)
# print(np.log10(msgr_Z),reffmsgr_Z)
# print(np.log10(msgi_Z),reffmsgi_Z)

In [15]:
# First we need Ie from Ltot on each band, then evaluate the stellar mass for each function, then the mass
# profile evaluated at each stellar mass profile

fileout = "test.txt"

Ie_g = []
Ie_i = []
Ie_r = []
Ms_gi = []
Ms_gr_B= []
Ms_gi_B= []
Ms_gr_Z = []
Ms_gi_Z= []
Re_ste_gi = []
Re_ste_gr_B= []
Re_ste_gi_B= []
Re_ste_gr_Z= []
Re_ste_gi_Z= []

Ie_g = np.zeros(603864)

for i in range(0, 603864):
    Ie_g[i] = Ltot_to_Ie(Ltot_g[i], Re_g[i], n_g[i])
#     Ie_g.append(Ltot_to_Ie(Ltot_g[i], Re_g[i], n_g[i]))
#     Ie_i.append(Ltot_to_Ie(Ltot_i[i], Re_i[i], n_i[i]))
#     Ie_r.append(Ltot_to_Ie(Ltot_r[i], Re_r[i], n_r[i]))
    
#     Ms_gi.append(Stellar_mass_from_gi(Ltot_i[i], Mg[i], Mi[i]))
#     Ms_gr_B.append(Stellar_mass_Bell_gr(Ltot_r[i], Mg[i], Mr[i]))
#     Ms_gi_B.append(Stellar_mass_Bell_gi(Ltot_i[i], Mg[i], Mi[i]))
#     Ms_gr_Z.append(Stellar_mass_Zibetti_gr(Ltot_r[i], Mg[i], Mr[i]))
#     Ms_gi_Z.append(Stellar_mass_Zibetti_gi(Ltot_i[i], Mg[i], Mi[i]))
    
#     Re_ste_gi.append(r_effective(Stellar_mass_from_gi, Ms_gi[i], Ie_g[i], Ie_i[i], n_g[i], n_i[i], Re_g[i], Re_i[i], Mgsol, Misol))
#     Re_ste_gr_B.append(r_effective(Stellar_mass_Bell_gr, Ms_gr_B[i], Ie_g[i], Ie_r[i], n_g[i], n_r[i], Re_g[i], Re_r[i], Mgsol, Mrsol))
#     Re_ste_gi_B.append(r_effective(Stellar_mass_Bell_gi, Ms_gi_B[i], Ie_g[i], Ie_i[i], n_g[i], n_i[i], Re_g[i], Re_i[i], Mgsol, Misol))
#     Re_ste_gr_Z.append(r_effective(Stellar_mass_Zibetti_gr, Ms_gr_Z[i], Ie_g[i], Ie_r[i], n_g[i], n_r[i], Re_g[i], Re_r[i], Mgsol, Mrsol))
# #     Re_ste_gi_Z.append(r_effective(Stellar_mass_Zibetti_gi, Ms_gi_Z[i], Ie_g[i], Ie_i[i], n_g[i], n_i[i], Re_g[i], Re_i[i], Mgsol, Misol))
#     print(i, Re_ste_gi[i])
    
    #np.savetxt(fileout, Ms_gi_Z, fmt = "%11.3f", comments='#')
    
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
# graph r_eff vs ms log10 both 

In [16]:

np.savetxt("archivo.txt", np.array([Ie_g, bla bla]).T) 

[32387642.74182985 45103558.85154735 28056376.7621882  ...
 13848707.82465864 72816082.40789722 72816082.40789722]
